In [1]:
%reset -f
%reload_ext autoreload
%autoreload 2
# %matplotlib inline
%config Completer.use_jedi = False

In [2]:
from pathlib import Path
from fastbook import *
from fastai.vision.all import *
from pyanis.fileops.utils import copy_files

# Cleaning with FastAI

In [3]:
def get_iw_info(
    learn,
    ds_idx:int=0 # Index in `learn.dls`
) -> list:
    "For every image in `dls` `zip` it's `Path`, target and loss"
    dl = learn.dls[ds_idx].new(shuffle=False, drop_last=False)
    probs,targs,preds,losses = learn.get_preds(dl=dl, with_input=False, with_loss=True, with_decoded=True)
    targs = [dl.vocab[t] for t in targs]
    return L([dl.dataset.items,targs,losses]).zip()

def label_func(fname):
    return "insect" if "/test/insect" in str(fname) or "/train/insect" in str(fname) else "not-insect"

In [4]:
# path = Path(f'/mnt/HGST_4TB/Datasets/insect_vs_noinsect/insect_vs_noinsect_copy/')
path = Path(f'/mnt/EE7455C074558BE9/backups/data_backups/images/insect_vs_noinsect/')

In [5]:
# CLEANING INVALID FILES
# fns = get_image_files(path)
# failed = verify_images(fns)
# assert not len(failed)
# failed.map(Path.unlink);

In [6]:
trials = 10
epochs = 6
th_loss = 2.5

for i in range(trials):
    insects = DataBlock(
        # provide a tuple where we specify what types we want for the independent and dependent variables:
        blocks=(ImageBlock, CategoryBlock), # What kinds of data we are working with
        get_items=get_image_files, # How to get the list of items
        splitter=RandomSplitter(valid_pct=0.2, seed=42), # How to create the validation set
        get_y=label_func, # How to label these items, gets the name of the folder a file is in
        item_tfms=Resize(150)) # Transformation: picture resize

    ds = insects.datasets(path)
    dls = insects.dataloaders(path) # the path where the images can be found
    # dls.train.show_batch(max_n=6, nrows=1) # check training set
    # dls.valid.show_batch(max_n=6, nrows=1) # check validation set
    # dls.show_batch(max_n=6, nrows=1) # check data

    print(f"Train data: {len(dls.train.dataset)}, Valid data: {len(dls.valid.dataset)}")
    learn = vision_learner(dls, densenet121, metrics=error_rate)
    learn.fine_tune(epochs)
    
    print("Gathering loss info..")
    iwis = get_iw_info(learn,0), get_iw_info(learn,1)
    df = pd.DataFrame(iwis[0]).sort_values(by=2, ascending=False)
    df1 = pd.DataFrame(iwis[1]).sort_values(by=2, ascending=False)
    df = pd.concat([df,df1])
    df[2] = df[2].apply(lambda x: float(x))

    print(f"Data to be moved: {len(df[df[2]>1.][0].tolist())}")
    copy_files(df[df[2]>th_loss][0].tolist(), '/mnt/EE7455C074558BE9/backups/data_backups/images/insect_vs_noinsect_review', move_files=True)

Train data: 73177, Valid data: 18294


epoch,train_loss,valid_loss,error_rate,time
0,0.111504,0.076258,0.028589,03:48


epoch,train_loss,valid_loss,error_rate,time
0,0.055076,0.037277,0.014486,05:22
1,0.043866,0.030981,0.011916,05:23
2,0.023726,0.028930,0.010222,05:23
3,0.010261,0.028320,0.009894,05:23
4,0.003067,0.019004,0.006286,05:23
5,0.002252,0.019054,0.005794,05:23


Gathering loss info..


Data to be moved: 87


...

epoch,train_loss,valid_loss,error_rate,time
0,0.053703,0.034692,0.014485,05:23
1,0.035329,0.022634,0.008998,05:24
2,0.021207,0.027412,0.010534,05:24
3,0.005842,0.012412,0.004664,05:24
4,0.002353,0.010071,0.002963,05:24
5,0.001220,0.010197,0.003182,05:23


Gathering loss info..


Data to be moved: 48


Moving files..: 100%|█████████████████████████| 23/23 [00:00<00:00, 3176.77it/s]

Finished copying files to:
	/mnt/EE7455C074558BE9/backups/data_backups/images/insect_vs_noinsect_review


In [24]:
# import matplotlib.pyplot as plt

# interp = ClassificationInterpretation.from_learner(learn)
# plt.figure(figsize=(20,12))
# interp.plot_confusion_matrix(figsize=(20,12))
# interp.plot_top_losses(10, nrows=4)

In [25]:
# from fastai.vision.widgets import *
# #hide_output
# cleaner = ImageClassifierCleaner(learn)

ImageClassifierCleaner doesn't actually do the deleting or changing of labels for you; it just returns the indices of items to change.


In [72]:
# cleaner

In [ ]:
# ImageClassifierCleaner doesn't actually do the deleting or changing of labels for you; it just returns the indices of items to change.

In [ ]:
# # To delete (unlink) all images selected for deletion, we would run:

# for idx in cleaner.delete(): cleaner.fns[idx].unlink()

# # To move images for which we've selected a different category, we would run:

# for idx,cat in cleaner.change(): shutil.move(str(cleaner.fns[idx]), path/cat)



In [ ]:
# df_hiloss = df[df[2]>2.5]

# plt.figure(figsize=(16,12));
# for i, (idx,path,target,loss) in enumerate(df_hiloss.itertuples()):
#     plt.subplot(9,9,i+1);
#     img = Image.open(path)
#     plt.imshow(img);plt.axis('off');plt.title(f"{target}/{loss:.2f}");